# preprocessing.ipynb

## Overview
This notebook is for cleaning and other data preprocessing of all the datasets in `/home/jovyan/ODBiz/1-PreProcessing/raw`

This notebook includes cells that do the following:
- Converts .shp files to .csv files
- Extracts lat/lon coordinates from cells that recorded them as JSON strings. Seems like the only dataset this applies to is `BC_Vancouver_Business_Licences.csv`
- Fixes `Indigenous_Business_Directory.csv`, which contained commas inside their cells
- Removes leading and trailing whitespaces from `NT_Yellowknife_Business_Directory.csv` and also fixes it's weirdly formatted phone numbers
- Standardizes dates for the `date_established` variables
- Moves processed datasets into the `2-OpenTabulate` folder

## External custom scripts
A few cells run external scripts have been written as .py files. Here's a list of links to those scripts:
- [process_shp_files.py](https://kubeflow.aaw.cloud.statcan.ca/notebook/deil-lode/odbiz-processing/doc/tree/ODBiz/1-PreProcessing/process_shp_files.py): Converts .shp files to .csv files
- [standardize_dates.py](https://kubeflow.aaw.cloud.statcan.ca/notebook/deil-lode/odbiz-processing/doc/tree/ODBiz/1-PreProcessing/standardize_dates.py): Standardizes dates for the `date_established` variables


In [1]:
# Import required packages
import os
from pathlib import Path
import shutil
import pandas as pd
import glob
import geopandas as gpd
import numpy as np

In [2]:
# First bring all raw files into processed. 
# We do this because more files won't need any cleaning work, so this is quicker than moving them manually. 
# Any files that require processing work will simply replace the raw files in the processed folder later

src = '../1-PreProcessing/raw'
dst = '../1-PreProcessing/processed'

if os.path.exists(dst):
    shutil.rmtree(dst)
    shutil.copytree(src, dst)
    
files_in_directory = os.listdir(dst)
filtered_files = [file for file in files_in_directory if not file.endswith(".csv")]
for file in filtered_files:
    try:
        path_to_file = os.path.join(dst, file)
        os.remove(path_to_file)
    except:
        pass

In [3]:
# Shapefile processing has been replaced with process_shp_files.py 
# This script includes Port Moody!
import process_shp_files
process_shp_files.main()

# # All shapefiles (except Port Moody)

# import os
# for root, dirs, files in os.walk("../1-PreProcessing/raw/shapefiles"):
#     for file in files:
#         if file.endswith(".shp"):
#             try:
#                 head, tail = os.path.split(os.path.join(root, file))
#                 head = head.replace('shapefile', '')
#                 head = head.replace("/home/jovyan/ODBiz/1-PreProcessing/raw/shapefiles/", '')
#                 head = head.replace("/", '')          
#                 tail = tail.replace('.shp', '')
#                 name = head + tail
#                 print(name)

#                 fp = (os.path.join(root, file))
#                 city = gpd.read_file(fp)
#                 print(city.crs)
#                 city = city.to_crs(epsg=4326)
#                 print(city.crs)
#                 sub_city = city.head(500)
#                 city['lon'] = city.geometry.x
#                 city['lat'] = city.geometry.y

#                 city.to_csv("../1-PreProcessing/raw/"+name+".csv")
#                 city.to_csv("../1-PreProcessing/processed/"+name+".csv")

#             except:
#                 print('error with file above')
#                 pass

BC_Port_Moody_Business_Directory
epsg:3857
epsg:4326
error with file above
BC_Burnaby_Business_Licences
epsg:4326

epsg:4326
gpd        -> pd                    | shared_chars
--------------------------------------------------
OBJECTID   -> OBJECTID              | 8
UNIT       -> UNIT                  | 4
PROPERTY_N -> PROPERTY_NUMBER       | 10
LICENCE_NU -> LICENCE_NUMBER        | 10
TRADE_NAME -> TRADE_NAME            | 10
LICENCE_TY -> LICENCE_TYPE_NAME     | 10
COVERS_FRO -> COVERS_FROM           | 10
COVERS_TO  -> COVERS_TO             | 9
ACCOUNT_NA -> ACCOUNT_NAME          | 10
HOUSE      -> HOUSE                 | 5
STREET     -> STREET                | 6
LEGACY_LIC -> LEGACY_LICENCE_NUMBER | 10
LEGAL_TYPE -> LEGAL_TYPE            | 10
LICENCE_ST -> LICENCE_STATUS        | 10
LGLLOT_PRO -> LGLLOT_PROPNUM        | 10
GEO_ID     -> GEO_ID                | 6
PARENT_PRO -> PARENT_PROPNUM        | 10

Unused columns from csv added to gdf:

File saved to /home/jovyan/ODBiz/1-PreProc

In [4]:
# # BC Port Moody Shapefile

# fp = "../1-PreProcessing/raw/shapefiles/BC_Port_Moody_shapefile/Business_Directory.shp"
# name = "port moody"

# city = gpd.read_file(fp)

# print(city.crs)
# city = city.to_crs(epsg=4326)
# print(city.crs)

# sub_city = city.head(500)

# city['lon'] = city.centroid.x
# city['lat'] = city.centroid.y

# #print(city.head)

# city.to_csv("../1-PreProcessing/raw/BC_Port_Moody_Business_Directory.csv")
# city.to_csv("../1-PreProcessing/processed/BC_Port_Moody_Business_Directory.csv")

In [5]:
# A lot of data cleaning had to be done for Vancouver, so this cell
# has been replaced with the process_vancouver.py script
import process_vancouver
process_vancouver.main()
# # BC vancouver lat/long

# df = pd.read_csv('../1-PreProcessing/raw/BC_Vancouver_Business_Licences.csv')

# def strip_point(x):   
#     try:
#         t = x.strip(r'{""coordinates"": [')
#         t = t.rstrip('], ""type"": ""Point""}')
#         t = t.replace(',', '')
#         return t.split()
#     except:
#         return np.nan

# LONGS=[]
# LATS=[]
# for i in df["Geom"]:
#     try:
#         LONGS.append(strip_point(i)[0])
#         LATS.append(strip_point(i)[1])
#     except:
#         LONGS.append(np.nan)
#         LATS.append(np.nan)

# df["long"]=LONGS
# df["lat"]=LATS

# df.to_csv('../1-PreProcessing/processed/BC_Vancouver_Business_Licences.csv')

Creating new business_name column: 100%|██████████| 636061/636061 [00:31<00:00, 20500.25it/s]
File saved to /home/jovyan/ODBiz/1-PreProcessing/processed/BC_Vancouver_Business_Licences.csv


In [6]:
# NT Yellowknife whitespaces and phone numbers

df = pd.read_csv('../1-PreProcessing/raw/NT_Yellowknife_Business_Directory.csv')

cols = ['BUSINESSNAME', 'MUNICIPAL ADDRESS3', 'BUSINESSTYPE', 'PHONE', 'EMAILADDRESS']
df[cols] = df[cols].apply(lambda x: x.str.strip())

df['PHONE'] = df['PHONE'].str.extract(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')

df.to_csv('../1-PreProcessing/processed/NT_Yellowknife_Business_Directory.csv')

In [7]:
# Fix the cells with commas in them in the Indigenous_Business_Directory.csv file

import csv

# Define csv file path
in_file = '../1-PreProcessing/raw/Indigenous_Business_Directory.csv'
out_file = '../1-PreProcessing/processed/Indigenous_Business_Directory.csv'

# Initialize a list that will be our new corrected csv
newcsv = []

# Open a read only copy of the csv file
with open(in_file, mode = 'r', newline='', encoding='utf8') as csvfile:
    csvreader = csv.reader(csvfile)

    # For each row in the csv...
    i = 0
    for row in csvreader:

        # For each cell in each row...
        j = 0
        for val in row:

            # If the known anomoly is found...
            if '$25' in val:

                # Then perform the fixing operation
                newval = ','.join(row[j:j+3])   # Concat the 3 cells that make up the sentence
                del row[j:j+3]                  # Delete the 3 cells from the list
                row.insert(j, newval)           # Insert the concat'd value back into the list
            j += 1

        # Append the row to the newcsv file
        newcsv.append(row)

        # Delete the unnecessary 19th column
        row_len = len(row)
        if row_len == 19:

            # Give a warning message if non-empty values are deleted
            if row[-1] != '':
                print('WARNING, DELETED VALUE:', row[-1])

            # Delete extra column
            del row[-1]

        # Add extra commas if they're missing
        row_len = len(row)
        print(i,':', row_len)
        while(len(row) < 18):
            row.append('')

        i += 1

# Delete the first row
del newcsv[0]

# Delete that one empty row after the header row
del newcsv[1]

# Save newcsv to a .csv file
with open(out_file, mode = 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    print(f'Saving the newcsv to {out_file} ...')
    writer.writerows(newcsv)
print(f'Saved newcsv to {out_file}')


WARNING, DELETED VALUE: Column19
0 : 18
1 : 18
2 : 18
3 : 18
4 : 18
5 : 18
6 : 18
7 : 18
8 : 18
9 : 18
10 : 18
11 : 18
12 : 18
13 : 18
14 : 18
15 : 18
16 : 18
17 : 18
18 : 18
19 : 18
20 : 18
21 : 18
22 : 18
23 : 18
24 : 18
25 : 18
26 : 18
27 : 18
28 : 18
29 : 18
30 : 18
31 : 18
32 : 18
33 : 18
34 : 18
35 : 18
36 : 18
37 : 18
38 : 18
39 : 18
40 : 18
41 : 18
42 : 18
43 : 18
44 : 18
45 : 18
46 : 18
47 : 18
48 : 18
49 : 18
50 : 18
51 : 18
52 : 18
53 : 18
54 : 18
55 : 18
56 : 18
57 : 18
58 : 18
59 : 18
60 : 18
61 : 18
62 : 18
63 : 18
64 : 18
65 : 18
66 : 18
67 : 18
68 : 18
69 : 18
70 : 18
71 : 18
72 : 18
73 : 18
74 : 18
75 : 18
76 : 18
77 : 18
78 : 18
79 : 18
80 : 18
81 : 18
82 : 17
83 : 18
84 : 18
85 : 18
86 : 18
87 : 18
88 : 18
89 : 18
90 : 18
91 : 18
92 : 18
93 : 18
94 : 18
95 : 18
96 : 18
97 : 18
98 : 18
99 : 18
100 : 18
101 : 18
102 : 18
103 : 18
104 : 18
105 : 18
106 : 18
107 : 18
108 : 18
109 : 18
110 : 18
111 : 18
112 : 18
113 : 18
114 : 18
115 : 17
116 : 18
117 : 18
118 : 18
119 : 

In [8]:
# Standardize the dates of the csvs with non-empty date_established fields using standardize_dates.py
import standardize_dates
standardize_dates.main()

Updated /home/jovyan/ODBiz/1-PreProcessing/processed/AB_Banff_Business_Licences.csv


In [9]:
# transfer files directly from PreProcessing/processed to opentabulate/data/input
src = '../1-PreProcessing/processed'
dst = '../2-OpenTabulate/data/input'

if os.path.exists(dst):
    shutil.rmtree(dst)
    shutil.copytree(src, dst)

In [10]:
# List the number of files(/folders?) in each directory listed below
raw = '../1-PreProcessing/raw'
pro = '../1-PreProcessing/processed'
input_ = '../2-OpenTabulate/data/input'
output_ = '../2-OpenTabulate/data/output'
print(len(os.listdir(raw)))
print(len(os.listdir(pro)))
print(len(os.listdir(input_)))
print(len(os.listdir(output_)))

63
63
63
61
